# 잡코리아 데이터 크롤링

## 라이브러리 선언

In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import re

## 함수 선언

In [2]:
def get_source(company: str) -> BeautifulSoup:
    """
    잡코리아에서 회사 이름에 대한 검색 결과 페이지를 반환하는 함수
    company: 잡코리아에서 검색할 회사 이름
    source: 잡코리아 회사 검색 결과 페이지, 함수 반환값
    """

    url = f'https://www.jobkorea.co.kr/Search/?stext={company}&tabType=corp'
    
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    return source

In [3]:
def get_company_url(source: BeautifulSoup) -> str:
    """
    잡코리아 회사 검색 결과 페이지에서 가장 상단에 있는 회사의 페이지 주소를 반환하는 함수
    source: 잡코리아 회사 검색 결과 페이지
    company_url: 잡코리아 회사 정보 페이지 주소, 반환값
    """

    corp_info = source.find('div', {'class': 'corp-info'})
    corp_list = corp_info.find('li', {'class': 'list-post'})
    corp_name = corp_list.find('a', {'class': 'name'})

    company_url = 'https://www.jobkorea.co.kr' + corp_name.get('href')

    return company_url

In [4]:
def get_assay_dict(company_url: str) -> BeautifulSoup:
    """
    잡코리아 합격자소서 페이지에서 자소서 목록을 반환하는 함수
    url: 잡코리아 회사 정보 페이지 주소
    assay_dict: {'자소서 제목': '자소서 페이지 번호'}, 반환값
    """

    assay_url = company_url + '/PassAssay'
    
    web = requests.get(assay_url).content
    source = BeautifulSoup(web, 'html.parser')

    assay_dict = dict()

    assay_div = source.find('div', {'class': 'passassayCont'})
    
    try:
        assay_list = assay_div.find_all('li', {'class': 'assay'})
    except AttributeError:
        return assay_dict

    for assay in assay_list:
        assay_title = assay.find('h2', {'class': 'tit'}).get_text().strip()
        assay_url = assay.find('a').get('href').split('&Part_Code=')[0]
        assay_number = assay_url.split('Job_Epil_No=')[1]
        assay_dict[assay_title] = assay_number

    return assay_dict

In [5]:
def get_assay_detail(company_url: str, assay_info: tuple) -> dict:
    """
    잡코리아 합격자소서 페이지의 모든 내용을 반환하는 함수
    assay_detail: 합격자소서 페이지의 모든 내용을 담은 딕셔너리, 반환값

    !!! 에러 발생 !!!
    합격자소서 페이지 요청 시 body가 자바스크립트 코드 안에 포함되어 반환
    클래스가 detailView인 article 태그를 찾아낼 수 없음
    """

    assay_title = assay_info[0]
    assay_number = assay_info[1]
    assay_detail = {'Title': assay_title}

    url = company_url + f'/PassAssay/View?Job_Epil_No={assay_number}'
    
    web = requests.get(url).content
    source = BeautifulSoup(web, 'html.parser')

    detail_view = source.find('article', {'class': 'detailView'})

    assay_detail['Index'] = get_assay_index(detail_view)

    qna_contents = get_qna_contents(detail_view)

    for i in range(len(qna_contents)):
        assay_detail[f'Q{i+1}'] = qna_contents[i]

    try:
        assay_detail['Advice'] = get_advice_contents(detail_view)
    except AttributeError:
        pass

    return assay_detail

In [6]:
def get_assay_index(detail_view: BeautifulSoup) -> list:
    """
    잡코리아 합격자소서 페이지에서 목차 목록을 반환하는 함수
    index_list: 합격자소서 페이지의 목차 목록, 반환값
    """

    index_div = detail_view.find('div', {'class': 'bx'})
    
    index_li = index_div.find_all('li')
    index_list = [index.get_text().strip() for index in index_li]
    
    return index_list

In [7]:
def get_qna_contents(detail_view: BeautifulSoup) -> list:
    """
    잡코리아 합격자소서 페이지에서 목차 목록을 반환하는 함수
    qna_contents: 합격자소서 페이지의 질문과 답변, 반환값
    """

    qna_div = detail_view.find('div', {'class': 'selfQnaWrap'})
    qna_titles = qna_div.find_all('dt', {'class': 'on'})
    qna_shows = qna_div.find_all('dd', {'class': 'show'})
    qna_contents = []

    for qna_title, qna_show in zip(qna_titles, qna_shows):
        qna_title = qna_title.find('span', {'class': 'tx'}).get_text()
        qna_lines = qna_show.find_all('b')
        if not qna_lines:
            qna_lines = qna_show.find('div', {'class': 'tx'})

        lines = []
        for qna_line in qna_lines:
            lines += re.split('\.\s', qna_line.get_text().strip())
        qna_lines = lines

        # qna_lines = []
        # for line in lines:
        #     if not any(x not in line for x in ['좋은점', '아쉬운점', '글자수']):
        #         qna_lines.append(line)
        
        qna_contents.append({qna_title: qna_lines})
    
    return qna_contents

In [8]:
def get_advice_contents(detail_view: BeautifulSoup) -> tuple:
    """
    잡코리아 합격자소서 페이지에서 총평 내용을 반환하는 함수
    advice_contents: 합격자소서 페이지의 총평 내용, 반환값
    """

    advice_div = detail_view.find('div', {'class': 'adviceTotal'})

    # advice_title = advice_div.find('h4', {'class': 'tit'}).get_text().strip()    
    advice_grade = advice_div.find('span', {'class': 'grade'}).get_text().strip()

    advice_lines = advice_div.find('p', {'class': 'tx'}).get_text()
    advice_lines = advice_lines.replace(' \r\n\r\n', ' ')
    advice_lines = advice_lines.replace('\r\n\r\n', ' ')
    advice_lines = re.split('\.\s', advice_lines)
    
    return advice_grade, advice_lines

In [9]:
def select_assay(original_dict: dict) -> tuple:
    """
    딕셔너리에서 하나의 키값을 선택해서 해당 키와 값을 반환하는 함수
    assay_dict: {'자소서 제목': '자소서 페이지 번호'}
    select_page: {'인덱스': ('자소서 제목', '자소서 페이지 번호')}, 반환값
    """
    
    assay_dict = original_dict.copy()
    index, selection = 1, None

    while True:
        select_page = dict()

        try:
            for i in range(10):
                select_page[index] = assay_dict.popitem()
                index += 1
        except KeyError:
            pass

        for idx, assay in select_page.items():
            print(f'{idx}. {assay[0]}')
        
        if assay_dict:
            print('다음 페이지로 이동(n)')
            selection = input('목록을 선택해주세요. ')
            if selection == 'n':
                continue
            return select_page[int(selection)]
        else:
            print('마지막 페이지 입니다.')
            selection = input('목록을 선택해주세요. ')
            return select_page[int(selection)]

In [10]:
def crawl_jobkorea():
    """
    잡코리아 크롤링 실행 함수
    현재 반환값: assay_detail
    """

    company = input('회사 이름을 입력하세요. ')

    try:
        source = get_source(company)
        company_url = get_company_url(source)
    except AttributeError or ConnectionError:
        raise Exception('회사 이름이 올바르지 않습니다.')
    
    assay_dict = get_assay_dict(company_url)

    if not assay_dict:
        print('합격자소서가 없습니다.')
        return None

    try:
        selection = select_assay(assay_dict)
    except KeyError or ValueError:
        raise Exception('잘못된 번호를 입력하셨습니다.')
    
    assay_detail = get_assay_detail(company_url, selection)
    print(assay_detail)
    
    # 작성 중

## 데이터 크롤링

In [12]:
try:
    crawl_jobkorea()
except Exception as e:
    print(e)
    print('다시 실행시켜 주세요.')

합격자소서가 없습니다.


## 테스트 코드

In [67]:
source = get_source('직방')
# source

In [68]:
company_url = get_company_url(source)
company_url

'https://www.jobkorea.co.kr/company/1866128'

In [73]:
assay_dict = get_assay_dict(company_url)
assay_dict

{'지원동기 및 입사후 포부': '192743', '[지원 동기: 함께하고 싶습니다]': '167157'}

In [55]:
import re

test_str = '/company/1623930/PassAssay/View?Job_Epil_No=200334&Part_Code=0&Search_Order=1&Page=1'

result = re.search('Job_Epil_No=(.*)&', test_str)
result[1]

'200334&Part_Code=0&Search_Order=1'

In [295]:
url = 'https://www.jobkorea.co.kr/company/1866128/PassAssay/View?Job_Epil_No=192743'

de = get_assay_detail('https://www.jobkorea.co.kr/company/1866128', ('지원동기 및 입사후 포부', '192743'))
de

{'Title': '지원동기 및 입사후 포부',
 'Index': ['Q1. 지원동기 및 입사후 포부', 'Q2. 성장과정', 'Q3. 성격의장단점'],
 'Q1': {'지원동기 및 입사후 포부': ['(주)직방은 골드만삭스의 3,300만 달러 투자로 부동산 O2O 기업에서의 가치를 인정받은 탄탄한 기업입니다',
   '더불어 헛걸음보상제, 매물광고 실명제 등을 실행함으로써 이용자들의 신뢰를 받아 부동산정보서비스 1위를 달성, 지켜가고 있습니다',
   '또한 집을 라이프스타일의 관점에서 바라보며 살 공간을 찾는 사람들의 시간과 비용을 줄여주어 더 편리하게 찾을 수 있도록 도와주고 있습니다',
   '첫째, 꼼꼼히 일하겠습니다',
   '(주)직방은 신뢰를 보내주는 많은 사람 덕분에 부동산서비스 어플 1위로 80%이상이 사용하고 있습니다',
   '자료 수집, 검수 등의 작업을 꼼꼼히 하여 믿을 수 있는 매물정보 제공, 믿을 수 있는 직방 이라는 목표를 달성하겠습니다',
   '두번째, 콘텐츠 기획을 위해 타운홀미팅을 적극 활용하겠습니다',
   '수요일에 열리는 전체 미팅에서 많은 지식과 아이디어를 배우고 직방이용자,임대인,부동산시장에서 필요한 콘텐츠를 기획하겠습니다',
   '마지막으로 매월 지급되는 자기계발비로 부동산 관련 지식 공부를 하는것에 투자하겠습니다',
   '또한 주택관리사 자격증을 1년이내에 취득하여 보다 전문적인 사원이 될것을 약속드립니다']},
 'Q2': {'성장과정': ['중학교 수련회때였습니다',
   '체력 극기 훈련 중이였는데 5단계를 모두 극복하면 상을 주겠다는 교관의 약속에 모든 학생이 체력 단련 훈련을 하고 있었습니다',
   '1단계, 2단계.',
   '단계가 올라갈수록 학생들은 난이도 높은 훈련에 포기를 선언했고 저는 4단계까지 버텼지만 5단계는 포기하기로 마음먹었습니다',
   '교관은 5단계로 가기를 포기한 사람들에게 말했습니다',
   ' "사실 이 훈련은 4단계가 끝입니다',
   '5단계에 도전하고자 남은 3명

[{'지원동기 및 입사후 포부': []},
 {'성장과정': ['노력해도 오르지 않던 수학점수도 "한번 더" 풀어보자, 다시 공식을 외워보자, 마음을 다잡으로 노력한 끝에 70점대에서 90점대로 올렸고, 대학생 때는 공모전 대회에서 100여명앞에서 발표해야 하는 큰 대회가 있었는데 많은사람들 앞에서 발표한다는 두려움도 "한번 더" 리허설 연습해보자, 대본 읽어보자, 말하기 연습하자 라고 다짐하여 성공적으로 수행할 수 있었습니다. 이러한 "한 번 더" 라는 마음이 지금의 제게 있어 시련과 어려움도 극복할 수 있는 용기가 되어주고 있습니다.  좋은점 1']}]

## 안 쓰는 함수

In [24]:
def get_company_page(source: BeautifulSoup) -> BeautifulSoup:
    """
    잡코리아 회사 검색 결과 페이지에서 가장 상단에 있는 회사의 페이지를 반환하는 함수
    source: 잡코리아 회사 검색 결과 페이지
    company_page: 잡코리아 회사 정보 페이지, 반환값
    """

    corp_info = source.find('div', {'class': 'corp-info'})
    corp_list = corp_info.find_all('li', {'class': 'list-post'})

    for corp in corp_list:
        try:
            corp_tag = corp.find('a', {'class': 'name'})
            corp_url = 'https://www.jobkorea.co.kr' + corp_tag.get('href')
        except AttributeError:
            continue
    
    web = requests.get(corp_url).content
    company_page = BeautifulSoup(web, 'html.parser')
    
    return company_page

In [25]:
def get_letter_page(url: BeautifulSoup) -> BeautifulSoup:
    """
    잡코리아 회사 정보 페이지에서 합격자소서 페이지를 반환하는 함수
    source: 잡코리아 회사 정보 페이지
    letter_page: 합격자소서 페이지, 반환값
    """

    company_nav = source.find('div', {'class': 'company-nav'})
    nav_items = company_nav.find_all('a', {'class': 'company-nav-item'})

    for nav_item in nav_items:
        if nav_item.get_text() == '합격자소서':
            letter_url = 'https://www.jobkorea.co.kr' + nav_item.get('href')
    
    web = requests.get(letter_url).content
    letter_page = BeautifulSoup(web, 'html.parser')
    
    return letter_page